In [1]:
import os, glob
import cv2
import numpy as np

In [3]:
# Paths
image_folder = './ODA_Dataset-master/calibration/camera/images/'  
output_file = './calibration_result.npz'

# Checkerboard parameters
checkerboard_size = (4, 7)
square_size = 0.024

objp = np.zeros((checkerboard_size[0] * checkerboard_size[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:checkerboard_size[0], 0:checkerboard_size[1]].T.reshape(-1, 2)
objp *= square_size

objpoints = []
imgpoints = []

# Load images
images = glob.glob(os.path.join(image_folder, '*.png'))

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    ret, corners = cv2.findChessboardCorners(gray, checkerboard_size, None)

    if ret:
        objpoints.append(objp)
        imgpoints.append(corners)
    else:
        print(f"Checkerboard not found in {fname}")

cv2.destroyAllWindows()

# Calibration
print("Running calibration...")
ret, K, dist, rvecs, tvecs = cv2.calibrateCamera(
    objpoints, imgpoints, gray.shape[::-1], None, None
)

print("Intrinsic matrix K =\n", K)
print("Distortion coefficients =\n", dist)

# Compute reprojection error
total_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], K, dist)
    error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2) / len(imgpoints2)
    total_error += error

mean_error = total_error / len(objpoints)
print("Mean reprojection error:", mean_error)

np.savez(output_file, K=K, distortion=dist)
print(f"Calibration data saved to {output_file}")


Running calibration...
Intrinsic matrix K =
 [[1.26524697e+03 0.00000000e+00 9.83931693e+02]
 [0.00000000e+00 1.25507591e+03 6.16590032e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion coefficients =
 [[ 0.01583852  0.06476464  0.01152856  0.00283166 -0.01495529]]
Mean reprojection error: 0.2473739988772376
Calibration data saved to ./calibration_result.npz
